In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses
import numpy as np
import matplotlib.pyplot as plt


In [2]:
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, axis=-1).astype('float32') / 255
x_test = np.expand_dims(x_test, axis=-1).astype('float32') / 255

# Simulating missing parts in images (inpainting mask)
def apply_mask(images, mask_width=10, mask_height=10):
    masked_images = images.copy()
    center_x = images.shape[1] // 2
    center_y = images.shape[2] // 2
    masked_images[:, center_x-mask_width//2:center_x+mask_width//2, center_y-mask_height//2:center_y+mask_height//2, :] = 0
    return masked_images

x_train_masked = apply_mask(x_train)
x_test_masked = apply_mask(x_test)


11490434/11490434 [==============================] - 0s 0us/step


In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

latent_dim = 32

# Encoder
encoder_inputs = layers.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

# Decoder
latent_inputs = layers.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation='relu')(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)
decoder = models.Model(latent_inputs, decoder_outputs, name='decoder')
decoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 14, 14, 32)           320       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 7, 7, 64)             18496     ['conv2d[0][0]']              
                                                                                                  
 flatten (Flatten)           (None, 3136)                 0         ['conv2d_1[0][0]']            
                                                                                            

In [4]:
class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed

vae = VAE(encoder, decoder)
vae.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [11]:
vae.fit(x_train_masked, x_train, epochs=70, batch_size=256, validation_data=(x_test_masked, x_test))


Epoch 1/70
235/235 [==============================] - 93s 396ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 2/70
235/235 [==============================] - 89s 380ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 3/70
235/235 [==============================] - 87s 370ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 4/70
235/235 [==============================] - 85s 362ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 5/70
235/235 [==============================] - 87s 371ms/step - loss: 0.0673 - val_loss: 0.0676
Epoch 6/70
235/235 [==============================] - 85s 362ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 7/70
235/235 [==============================] - 86s 367ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 8/70
235/235 [==============================] - 86s 368ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 9/70
235/235 [==============================] - 87s 369ms/step - loss: 0.0673 - val_loss: 0.0675
Epoch 10/70
235/235 [==============================] - 85s 364ms/step - l

In [ ]:
def plot_results(models, data, batch_size=128, model_name="vae_mnist"):
    encoder, decoder = models
    x_test, y_test = data
    z_mean, _, _ = encoder.predict(x_test, batch_size=batch_size)
    reconstructed = decoder.predict(z_mean)
    plt.figure(figsize=(20, 4))
    for i in range(10):
        # Display original
        ax = plt.subplot(2, 10, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.title("Original")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display reconstruction
        ax = plt.subplot(2, 10, i + 1 + 10)
        plt.imshow(reconstructed[i].reshape(28, 28))
        plt.title("Reconstructed")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

plot_results((encoder, decoder), (x_test_masked, x_test))